In [2]:

import pandas as pd
import numpy as np

# Ruta del archivo en tu equipo
file_path = r"C:\Users\zoebe\OneDrive\Escritorio\bi analyst\Hendylaset.xlsx"

# Cargar las hojas
ordenes_df = pd.read_excel(file_path, sheet_name='ordenes')
cart_items_df = pd.read_excel(file_path, sheet_name='cart-items')

In [4]:
# --- LIMPIEZA DE ORDENES ---

# Corregir duplicados en 'order_id'
duplicados = ordenes_df[ordenes_df['order_id'].duplicated(keep=False)].copy()
duplicados['recuento'] = duplicados.groupby('order_id').cumcount() + 1
duplicados['order_id'] = duplicados.apply(
    lambda row: f"{row['order_id']}_{row['recuento']}" if row['recuento'] > 1 else row['order_id'],
    axis=1
)
duplicados.drop(columns='recuento', inplace=True)
for idx, row in duplicados.iterrows():
    ordenes_df.loc[idx, 'order_id'] = row['order_id']

# Eliminar filas donde 'cart_id' no existe en cart-items
cart_items_valid_ids = set(cart_items_df['cart_id'])
ordenes_df = ordenes_df[ordenes_df['cart_id'].isin(cart_items_valid_ids)]

# Agrupar por 'cart_id' como clave única
ordenes_df = ordenes_df.groupby('cart_id', as_index=False).first()

In [5]:
# --- LIMPIEZA DE CART-ITEMS ---

# Reemplazar valores nulos en 'category' con "Null"
cart_items_df['category'] = cart_items_df['category'].fillna("Null")

# Crear jerarquía de categorías
category_split = cart_items_df['category'].str.split('>', expand=True)
category_split.columns = [f'Categoria_Nivel_{i+1}' for i in range(category_split.shape[1])]
category_split = category_split.apply(lambda col: col.str.strip())
cart_items_df = pd.concat([cart_items_df, category_split], axis=1)

# Limpiar shipping_type
valores_validos = ['standard', 'rapido', 'lento', 'pickup']
cart_items_df['shipping_type'] = cart_items_df['shipping_type'].str.strip().str.lower()
cart_items_df['shipping_type'] = cart_items_df['shipping_type'].apply(
    lambda x: x if x in valores_validos else np.nan
)

# Validar 'quantity'
cart_items_df['quantity'] = pd.to_numeric(cart_items_df['quantity'], errors='coerce')

# Validar 'manufacturer'
cart_items_df['manufacturer'] = cart_items_df['manufacturer'].apply(
    lambda x: x if isinstance(x, str) else np.nan
)

In [6]:
# --- FILTRAR ORDENES POR AÑO 2013 ---
ordenes_df['timestamp_created'] = pd.to_datetime(ordenes_df['timestamp_created'], errors='coerce')
ordenes_df = ordenes_df[ordenes_df['timestamp_created'].dt.year == 2013]

# --- FILTRAR CART ITEMS POR LOS CART_ID DE 2013 ---
cart_ids_2013 = set(ordenes_df['cart_id'])
cart_items_df = cart_items_df[cart_items_df['cart_id'].isin(cart_ids_2013)]



In [7]:
# --- EXPORTAR A EXCEL ---

output_path = r"C:\\Users\\zoebe\\OneDrive\\Escritorio\\bi analyst\\Hendylaset_limpio.xlsx"
with pd.ExcelWriter(output_path) as writer:
    ordenes_df.to_excel(writer, sheet_name='ordenes_2013', index=False)
    cart_items_df.to_excel(writer, sheet_name='cart-items_limpio', index=False)

print(f"✅ Archivo exportado exitosamente a:\n{output_path}")

✅ Archivo exportado exitosamente a:
C:\\Users\\zoebe\\OneDrive\\Escritorio\\bi analyst\\Hendylaset_limpio.xlsx
